In [8]:
from main import client, logger, INGEST_PATH, TARGET_PATH
from tqdm import tqdm
from glob import glob
import pandas as pd
import os

In [9]:
def load_hdf(file, entry_point):
    df = pd.read_hdf(file, 'df')
    df['file_name'] = file[len(entry_point)+1:]
    
    #Join neural data into an array data type (dimensionality may vary)
    #They are the numerical columns, so filter by that.
    numerical_columns = list(filter(lambda x: isinstance(x, int), df.columns))
    
    df["neural_data"] = df[numerical_columns].values.copy().tolist()
    df = df.drop(columns = numerical_columns)
    
    return df

In [11]:
for file in tqdm(glob(f"{INGEST_PATH}/**/*.h5", recursive = True)):
    try:
        df = load_hdf(file, INGEST_PATH)
        save_path = f"{TARGET_PATH}/{file[len(INGEST_PATH)+1:]}"
        if not os.path.exists(save_path):
            dir_name = os.path.dirname(save_path)
            os.makedirs(dir_name, exist_ok=True)
        save_path = os.path.splitext(save_path)[0] + ".parquet"
        df.to_parquet(save_path)
    except Exception as e:
        logger.error(f"Error processing {file}: {e}")

  0%|          | 0/52 [00:00<?, ?it/s]

 48%|████▊     | 25/52 [08:17<10:33, 23.46s/it]2024-12-09 14:17:32,513 - main - ERROR - Error processing /home/icaro/Documents/doctorate/raw_data/ID18150/Day3/DataFrame_Imaging_dFF_18150_day3.h5: 'NoneType' object has no attribute 'startswith'
2024-12-09 14:17:32,513 - main - ERROR - Error processing /home/icaro/Documents/doctorate/raw_data/ID18150/Day3/DataFrame_Imaging_dFF_18150_day3.h5: 'NoneType' object has no attribute 'startswith'
100%|██████████| 52/52 [17:19<00:00, 19.99s/it]
